# setup

See the official repository for details.
- [how to build · Tencent/ncnn Wiki](https://github.com/Tencent/ncnn/wiki/how-to-build)
- [ncnn/release\.yml at master · Tencent/ncnn](https://github.com/Tencent/ncnn/blob/master/.github/workflows/release.yml)

In [1]:
!git clone https://github.com/Tencent/ncnn.git -b 20220216 --depth 1
%cd ncnn
!git submodule update --init

Cloning into 'ncnn'...
remote: Enumerating objects: 2497, done.
remote: Counting objects: 100% (2497/2497), done.
remote: Compressing objects: 100% (1527/1527), done.
remote: Total 2497 (delta 1829), reused 1194 (delta 963), pack-reused 0
Receiving objects: 100% (2497/2497), 7.52 MiB | 6.27 MiB/s, done.
Resolving deltas: 100% (1829/1829), done.
Note: checking out '6b2495cc243f2d8e829523b700f32db1f5d50f78'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by performing another checkout.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -b with the checkout command again. Example:

  git checkout -b <new-branch-name>

/content/ncnn
Submodule 'glslang' (https://github.com/KhronosGroup/glslang) registered for path 'glslang'
Submodule 'python/pybind11' (https://github.com/pybind/pybind11.git) registered 

## android


Download the ndk and set the path

In [2]:
!wget https://dl.google.com/android/repository/android-ndk-r23b-linux.zip
!unzip -q android-ndk-r23b-linux.zip
%env ANDROID_NDK=/content/ncnn/android-ndk-r23b

!sed -i -e '/^  -g$/d' $ANDROID_NDK/build/cmake/android.toolchain.cmake

--2022-02-22 02:39:46--  https://dl.google.com/android/repository/android-ndk-r23b-linux.zip
Resolving dl.google.com (dl.google.com)... 142.251.31.93, 142.251.31.136, 142.251.31.190, ...
Connecting to dl.google.com (dl.google.com)|142.251.31.93|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 725122099 (692M) [application/zip]
Saving to: ‘android-ndk-r23b-linux.zip’

android-ndk-r23b-li 100%[===================>] 691.53M   264MB/s    in 2.6s    

2022-02-22 02:39:49 (264 MB/s) - ‘android-ndk-r23b-linux.zip’ saved [725122099/725122099]

env: ANDROID_NDK=/content/ncnn/android-ndk-r23b


### Build ncnn

In [3]:
!pip install cmake --upgrade
!cmake --version

     |████████████████████████████████| 22.7 MB 71.6 MB/s 
  Attempting uninstall: cmake
    Found existing installation: cmake 3.12.0
    Uninstalling cmake-3.12.0:
      Successfully uninstalled cmake-3.12.0
cmake version 3.22.2

CMake suite maintained and supported by Kitware (kitware.com/cmake).


I don't want to bundle OpenCV with my app because it will increase the size of my app.

By building with the `-DNCNN_SIMPLEOCV=ON` flag, OpenCV will be unnecessary.

Also, set `-DNCNN_VULKAN=OFF` since Vulkan is not used in this build.

In [4]:
!mkdir -p build-android-armv7
%cd build-android-armv7

!cmake \
  -DCMAKE_TOOLCHAIN_FILE=$ANDROID_NDK/build/cmake/android.toolchain.cmake \
  -DCMAKE_BUILD_TYPE=Release \
  -DCMAKE_INSTALL_PREFIX=install \
  -DANDROID_ABI="armeabi-v7a" \
  -DANDROID_ARM_NEON=ON \
  -DNCNN_VULKAN=OFF \
  -DANDROID_PLATFORM=android-24 \
  -DNCNN_SIMPLEOCV=ON \
  -DNCNN_BUILD_BENCHMARK=OFF \
  ..

!make -j$(nproc)
!make install

%cd ../

!mkdir -p build-android-aarch64
%cd build-android-aarch64
!cmake \
  -DCMAKE_TOOLCHAIN_FILE="$ANDROID_NDK/build/cmake/android.toolchain.cmake" \
  -DCMAKE_BUILD_TYPE=Release \
  -DCMAKE_INSTALL_PREFIX=install \
  -DANDROID_ABI="arm64-v8a" \
  -DNCNN_VULKAN=OFF \
  -DANDROID_PLATFORM=android-24 \
  -DNCNN_SIMPLEOCV=ON \
  -DNCNN_BUILD_BENCHMARK=OFF \
  ..
!make -j$(nproc)
!make install

/content/ncnn/build-android-armv7
-- CMAKE_TOOLCHAIN_FILE = /content/ncnn/android-ndk-r23b/build/cmake/android.toolchain.cmake
-- CMAKE_INSTALL_PREFIX = install
-- NCNN_VERSION_STRING = 1.0.20220222
-- Android: Targeting API '24' with architecture 'arm', ABI 'armeabi-v7a', and processor 'armv7-a'
-- Android: Selected unified Clang toolchain
-- The C compiler identification is Clang 12.0.8
-- The CXX compiler identification is Clang 12.0.8
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /content/ncnn/android-ndk-r23b/toolchains/llvm/prebuilt/linux-x86_64/bin/clang - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /content/ncnn/android-ndk-r23b/toolchains/llvm/prebuilt/linux-x86_64/bin/clang++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Performing

In [5]:
# zip it and save it to Google Drive
!zip -q -r /content/drive/MyDrive/ncnn-build-android.zip /content/ncnn/build-android-armv7 /content/ncnn/build-android-aarch64